In [ ]:
#from google.colab import files
#uploaded = files.upload()  # Te mostrará un botón para seleccionar el archivo Video_Games.csv

Saving Video_Games.csv to Video_Games.csv


In [1]:
import numpy as np
import pandas as pd
import time
from tqdm.notebook import tqdm
from keras.models import Model
from keras.layers import Embedding, Flatten, Input, Dense, Concatenate, Dot
from keras.regularizers import l2
from keras.layers import Add
from sklearn.metrics import mean_absolute_error

ModuleNotFoundError: No module named 'keras'

In [16]:
df = pd.read_csv('Video_Games.csv', header=None, nrows=100000)
df.columns = ['item_id', 'user_id', 'rating', 'timestamp']
df = df.drop('timestamp', axis=1)
print(f"Tamaño total del dataset: {df.shape}")

Tamaño total del dataset: (100000, 3)


In [17]:
user_ids = df['user_id'].unique()
item_ids = df['item_id'].unique()
NUM_USERS = len(user_ids)
NUM_ITEMS = len(item_ids)
print(f"NUM_USERS = {NUM_USERS}, NUM_ITEMS = {NUM_ITEMS}")

NUM_USERS = 71999, NUM_ITEMS = 2531


In [18]:
user2idx = {user_id: i for i, user_id in enumerate(user_ids)}
item2idx = {item_id: i for i, item_id in enumerate(item_ids)}

In [21]:
df = df.sample(frac=1, random_state=42)
split_index = int(len(df) * 0.8)
train_df = df[:split_index]
test_df = df[split_index:]

In [22]:
user_indices_train = np.zeros(len(train_df), dtype=np.int32)
item_indices_train = np.zeros(len(train_df), dtype=np.int32)
ratings_train = np.zeros(len(train_df), dtype=np.float32)

for idx, (_, row) in enumerate(tqdm(train_df.iterrows(), total=len(train_df))):
    user_indices_train[idx] = user2idx[row['user_id']]
    item_indices_train[idx] = item2idx[row['item_id']]
    ratings_train[idx] = row['rating']

X_train = [user_indices_train, item_indices_train]
y_train = ratings_train

Preparando datos de entrenamiento...


  0%|          | 0/80000 [00:00<?, ?it/s]

In [23]:
user_indices_test = np.zeros(len(test_df), dtype=np.int32)
item_indices_test = np.zeros(len(test_df), dtype=np.int32)
ratings_test = np.zeros(len(test_df), dtype=np.float32)

for idx, (_, row) in enumerate(tqdm(test_df.iterrows(), total=len(test_df))):
    user_indices_test[idx] = user2idx[row['user_id']]
    item_indices_test[idx] = item2idx[row['item_id']]
    ratings_test[idx] = row['rating']

X_test = [user_indices_test, item_indices_test]
y_test = ratings_test

Preparando datos de prueba...


  0%|          | 0/20000 [00:00<?, ?it/s]

GMF

In [42]:
latent_dim = 20
epochs = 10
reg_factor = 0.01

In [43]:
rating_min = y_train.min()
rating_max = y_train.max()
y_train_normalized = (y_train - rating_min) / (rating_max - rating_min)

In [62]:
user_input = Input(shape=(1,))
user_embedding = Embedding(NUM_USERS, latent_dim,
                          embeddings_regularizer=l2(reg_factor))(user_input)
user_vec = Flatten()(user_embedding)

item_input = Input(shape=(1,))
item_embedding = Embedding(NUM_ITEMS, latent_dim,
                          embeddings_regularizer=l2(reg_factor))(item_input)
item_vec = Flatten()(item_embedding)

output = Dot(axes=1)([user_vec, item_vec])

user_bias = Embedding(NUM_USERS, 1,
                     embeddings_regularizer=l2(reg_factor))(user_input)
item_bias = Embedding(NUM_ITEMS, 1,
                     embeddings_regularizer=l2(reg_factor))(item_input)
user_bias = Flatten()(user_bias)
item_bias = Flatten()(item_bias)

output = Add()([output, user_bias, item_bias])

MF = Model([user_input, item_input], output)

In [47]:
MF.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mean_absolute_error']
)

MF.summary()

MF.fit(
    [X_train[0], X_train[1]],
    y_train_normalized,
    epochs=epochs,
    batch_size=256,
    validation_split=0.1,
    verbose=1
)

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_12      │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_13      │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_14        │ (None, 1, 20)     │  1,439,980 │ input_layer_12[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_15        │ (None, 1, 20)     │     50,620 │ input_layer_13[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_14          │ (None, 20)        │          0 │ embedding_14[0][… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_15          │ (None, 20)        │          0 │ embedding_15[0][… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_16        │ (None, 1, 1)      │     71,999 │ input_layer_12[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_17        │ (None, 1, 1)      │      2,531 │ input_layer_13[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot_6 (Dot)         │ (None, 1)         │          0 │ flatten_14[0][0], │
│                     │                   │            │ flatten_15[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_16          │ (None, 1)         │          0 │ embedding_16[0][… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_17          │ (None, 1)         │          0 │ embedding_17[0][… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 1)         │          0 │ dot_6[0][0],      │
│                     │                   │            │ flatten_16[0][0], │
│                     │                   │            │ flatten_17[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,565,130 (5.97 MB)

 Trainable params: 1,565,130 (5.97 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
282/282 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 0.6329 - mean_absolute_error: 0.6906 - val_loss: 0.6395 - val_mean_absolute_error: 0.6930
Epoch 2/10
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.6340 - mean_absolute_error: 0.6882 - val_loss: 0.6396 - val_mean_absolute_error: 0.6926
Epoch 3/10
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.6352 - mean_absolute_error: 0.6886 - val_loss: 0.6397 - val_mean_absolute_error: 0.6927
Epoch 4/10
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.6336 - mean_absolute_error: 0.6876 - val_loss: 0.6397 - val_mean_absolute_error: 0.6925
Epoch 5/10
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.6346 - mean_absolute_error: 0.6886 - val_loss: 0.6397 - val_mean_absolute_error: 0.6926
Epoch 6/10
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.6332 - mean_absolute_error: 0.6873 - val_loss: 0.6397 - val_mean_absolute_error: 0.6928
Epoch 7/10
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.6324 - mean_absolute_error: 0.6867 - v

In [49]:
y_pred = MF.predict(X_test)
y_pred

625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


array([[0.03808517],
       [0.38036162],
       [0.06119625],
       ...,
       [0.06157325],
       [0.10874471],
       [0.1711136 ]], dtype=float32)

In [50]:
mean_absolute_error(y_test, y_pred)

4.064741611480713

MLP

In [52]:
latent_dim = 5
epochs = 10
reg_factor = 0.01

In [53]:
rating_min = y_train.min()
rating_max = y_train.max()
y_train_normalized = (y_train - rating_min) / (rating_max - rating_min)
y_test_normalized = (y_test - rating_min) / (rating_max - rating_min)

In [54]:
user_input = Input(shape=(1,))
user_embedding = Embedding(NUM_USERS, latent_dim,
                          embeddings_regularizer=l2(reg_factor))(user_input)
user_vec = Flatten()(user_embedding)

item_input = Input(shape=(1,))
item_embedding = Embedding(NUM_ITEMS, latent_dim,
                          embeddings_regularizer=l2(reg_factor))(item_input)
item_vec = Flatten()(item_embedding)

concat = Concatenate()([user_vec, item_vec])

dense1 = Dense(20, "relu")(concat)
dense2 = Dense(10, "relu")(dense1)
output = Dense(1, "relu")(dense2)

user_bias = Embedding(NUM_USERS, 1,
                     embeddings_regularizer=l2(reg_factor))(user_input)
item_bias = Embedding(NUM_ITEMS, 1,
                     embeddings_regularizer=l2(reg_factor))(item_input)

user_bias = Flatten()(user_bias)
item_bias = Flatten()(item_bias)

output_with_bias = Add()([output, user_bias, item_bias])

MLP = Model([user_input, item_input], output_with_bias)

In [55]:
MLP.compile(
    optimizer="adam",
    metrics=["mae"],
    loss="mean_squared_error"
)

MLP.summary()

MLP.fit(X_train, y_train_normalized, epochs=epochs, verbose=1)

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_16      │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_17      │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_22        │ (None, 1, 5)      │    359,995 │ input_layer_16[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_23        │ (None, 1, 5)      │     12,655 │ input_layer_17[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_22          │ (None, 5)         │          0 │ embedding_22[0][… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_23          │ (None, 5)         │          0 │ embedding_23[0][… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 10)        │          0 │ flatten_22[0][0], │
│ (Concatenate)       │                   │            │ flatten_23[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 20)        │        220 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 10)        │        210 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_24        │ (None, 1, 1)      │     71,999 │ input_layer_16[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_25        │ (None, 1, 1)      │      2,531 │ input_layer_17[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 1)         │         11 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_24          │ (None, 1)         │          0 │ embedding_24[0][… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_25          │ (None, 1)         │          0 │ embedding_25[0][… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 1)         │          0 │ dense_5[0][0],    │
│                     │                   │            │ flatten_24[0][0], │
│                     │                   │            │ flatten_25[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 447,621 (1.71 MB)

 Trainable params: 447,621 (1.71 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.2880 - mae: 0.2990
Epoch 2/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.1207 - mae: 0.2384
Epoch 3/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 0.1216 - mae: 0.2363
Epoch 4/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 0.1189 - mae: 0.2318
Epoch 5/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 0.1192 - mae: 0.2315
Epoch 6/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 0.1202 - mae: 0.2309
Epoch 7/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 0.1220 - mae: 0.2327
Epoch 8/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 0.1215 - mae: 0.2306
Epoch 9/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 0.1210 - mae: 0.2294
Epoch 10/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 0.1216 - mae: 0.2308


In [61]:
y_pred = MLP.predict(X_test)
y_pred

625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


array([[0.8617932 ],
       [0.79479957],
       [0.84403944],
       ...,
       [0.8687748 ],
       [0.65952575],
       [0.8526762 ]], dtype=float32)

In [60]:
mean_absolute_error(y_test_normalized, y_pred)


0.23007719218730927